# Applying MCMC to asset pricing models

This is a self-started project I wrote after learning the famous Markov chain Monte Carlo method in Bayesian statistics. I followed a few academic papers on the topic and tried to summerize the problems for the applications to asset price models and how I can be solved. This is a technical paper and it is revolved arounf the mathematics behind the method. Therefore, no data will be used.

# Abstract

It is assumed that we don't know and can't predict future values of asset prices. Although we cannot forecast asset prices via its past values, we can obtain useful information such as mean, variance, likely distribution by building asset pricing models. This is studied in the dynamic asset pricing theory. In the option pricing theory, we learned about how to price options by solving the Black-Scholes model (a stochastic differential equation).

Empirical analysis of dynamic asset pricing models focuses on extracting information about the latent variables (e.g personal believes), structural parameters and market prices of risk from observed prices. Namely, we aim to make inference about the distribution of parameters $\Theta$, state variables $X$ conditioned on observed price $Y$. The Bayesian approach on this problem is to make inference on the posterior distribution $p(\Theta, X | Y)$.

This  project aims to review on applying MCMC method to generate samples from $p(\Theta, X|Y)$ by asset pricing models. The approach first follow Duffie(1996), then interpret asset pricing models as state space models.

# Problems for some mthods in Finance

In this section, We talk about what is the problems for most mthods and why MCMC method is good for the models. Recall fromabstract we wish to make inference on $p(\Theta|X,Y)$.  However, $p(\Theta|X,Y)$ can be extremely complicated and high-dimensional. The Clifford-Hammersley theorem states that the breaking of $p(\Theta|X,Y)$ into its complete conditional(the conditional given other variables) distributions is theoretically justified. Therefore, we can use MCMC to sample the broken conditional sets, which has lower dimension and are easier to simple. This is what MCMC has advantage over other methods. For example, lets assume that $\Theta = (\Theta_1, \dots, \Theta_n)$ is a $n$-dimensional vector and it can be partitioned into $k \leq n$ components $\Theta = (\theta_1, \dots, \theta_k)$, where each $\theta_i$ can be either 1-dimentional or multidimensional. Then the Clifford-Hammersley theorem states that the following set of conisitional distributions


\begin{split}
\theta_1 & | \theta_2, \theta_2, \dots, X, Y \\
\theta_2 & | \theta_1, \theta_3, \dots, X, Y \\
\vdots \\
\theta_k & | \theta_1, \theta_2, \dots, \theta_{k-1}, X, Y
\end{split}

uniquely determines $p(\Theta|X,Y)$. For the state vector $X$, the joint distribution $p(X|\Theta, Y)$ can be characterized by its own complete set of conditional: $p(X_t| \Theta, X_{(-t)},Y)$ for $t = 1,\dots, T$ where $X_{(-t)}$ is the elements of $X$ excluding $X_t$. If we use the Gibbs sampler, the process will be as follows: given $(\Theta^{(0)}, X^{(0)})$


\begin{split}
1. & \text{ Draw }\Theta^{(1)}_1 \sim p(\Theta_1 | \Theta^{(0)}_2, \Theta^{(0)}_3, \dots, \Theta^{(0)}_n, X^{(0)}, Y) \\
2. & \text{ Draw }\Theta^{(1)}_2 \sim p(\Theta_2 | \Theta^{(0)}_1, \Theta^{(0)}_3, \dots, \Theta^{(0)}_n, X^{(0)}, Y) \\
\vdots \\
n. & \text{ Draw }\Theta^{(1)}_n \sim p(\Theta_n | \Theta^{(0)}_1, \Theta^{(0)}_2, \dots, \Theta^{(0)}_{n-1}, X^{(0)}, Y) \\
\end{split}


and then draw the states $p(X|\Theta, Y)$. If the states cannot be drawn in one of the black, then by the same arguement, we can factor $p(X|\Theta, Y)$ into a set of lower dimensional distributions.

# Geometric Brownian Motion

In my original paper, we have obtained a closed form solution for the Black-Scholes price model:

\begin{equation}
X_t = \log(\frac{S_t}{S_{t-1}}) = \mu + \varepsilon_{t}
\end{equation}

Here the model generates a conitional likelihood for the vector of continuously-compounded returns of

\begin{equation}
p(Y|\mu, \sigma^2) = \left( \frac{1}{\sqrt{2 \pi \sigma^2}} \right)^T \exp \left( -\frac{1}{2 \sigma^2} \sum^T_{t=1} (Y_t - \mu)^2 \right)
\end{equation}

where $Y = (S_0, \dots, S_T)$ is the vector of asset price of in the time interval $(0, \dots, T)$. Here we wish to make inference on $p(\Theta|Y)$. Since there are no latent variables in the model, this implies $\Theta = (\mu, \sigma^2)$ and $p(\Theta | Y) = p(\mu, \sigma^2 | Y)$.

In my orginal paper, we have a price formula for the price of a call option stuck at $K$:

\begin{equation}
C_t = BS(\sigma, S_t) = S_t \Phi(d_1) - e^{r(T-t)}K \Phi(d_1 - \sigma \sqrt{T-t}) 
\end{equation}

where $\Phi(x) = \frac{1}{2} - \frac{1}{2} erf \left( \frac{x-\mu}{\sigma \sqrt{2}} \right)$ and 

\begin{equation}
d_1 = \frac{\log(S_t / K) + (r + \sigma^2 / 2 (T-t)}{\sigma \sqrt{T-t}}
\end{equation}

Assume that option prices are observed with a normally distributed error, then the state space model is

\begin{split}
& \log(S_t/S_{t-1}) = \mu + \sigma \varepsilon_{t} \\
& C_t = BS(\sigma, S_t) + \varepsilon^c_{t}
\end{split}

where $\varepsilon_{t} \sim \mathcal{N}(0,1)$ and $\varepsilon^c_{t} \sim \mathcal{N}(0, \sigma^2_c)$. This model is conditional normal, but there is no analytical solution. We would like to make inference on $\mu,\sigma^2$ and $\sigma^2_c$. The joint likelihood function is the product of the equity return likelihood, $p(Y|\mu, \sigma^2)$, and the option likelihood, $p(C|S, \mu, \sigma^2, \sigma^2_c)$. That is,

\begin{equation}
p(S,C|\mu, \sigma^2, \sigma^2_c) = \prod^T_{t=1} p(C_t|S_t, \sigma^2, \sigma^2_c)p(\log(S_t/S_{t-1})|\mu,\sigma)
\end{equation}

where $S = (S_1, \dots, S_T)$ is the vector of  prices of underlying asset and $C = (C_1, \dots, C_T)$ is the vector of prices of the option. As mentioned before, the option price component of the likelihood is normal and is given by

\begin{equation}
p(C_t|S_t,\sigma,\sigma^2_c) \propto \exp \left(-\frac{1}{2 \sigma^2_c}(C_t - BS(\sigma, S_t))^2 \right)
\end{equation}

The option price conditional distribution is independent of $\mu$ and the stock returns distribution is independent of $\sigma^2_c$. The MCMC algorithm samples from the joint posterior, $p(\mu, \sigma^2, \sigma^2_c|S,C)$. The complete conditionals are $p(\mu| \sigma^2, S), p(\sigma^2|\mu, \sigma^2_c,S,C)$ and $p(\sigma^2_c|\sigma^2,S,C)$. In the Bayesian course, we know that the conjugate priors are: $p(\mu) \sim \mathcal{N}$, $p(\sigma^2) \sim \mathcal{IG}$ and $p(\sigma^2_c) \sim \mathcal{IG}$. The MCMC algorithm collects samples as follows:

\begin{split}
\mu^{(g+1)} & \sim p \left( \mu | (\sigma^2)^{(g)},S \right) \sim \mathcal{N} \\
(\sigma^2_c)^{(g+1)} & \sim p \left( \sigma^2_c | (\sigma^2)^{(g)},S,C \right) \sim \mathcal{IG} \\
(\sigma^2)^{(g+1)} & \sim p \left( \sigma^2 | \mu^{(g+1)}, (\sigma^2_c)^{(g+1)},S,C \right)
\end{split}

Notice that 

\begin{equation}
p \left( C_t|S_t,\sigma^2,\sigma^2_c \right) \propto \exp \left( -\frac{1}{2 \sigma^2_c}(C_t - BS(\sigma,S_t))^2 \right)
\end{equation}

but $BS(\sigma,S(t)$ is given as an integral (erf is the error function and is with an integral). Therefore, it is impossible to sample directly from $p(\sigma^2 | \mu, \sigma^2_c,S,Y)$. In this case, we are using the independence Metropolis-Hasting algoriothm. Namely, consider a proposal of the form

\begin{equation}
q(\sigma^2) = p(\sigma^2|\mu,S) \propto p(S|\mu,\sigma)p(\sigma^2) \sim \mathcal{IG}
\end{equation}

The algorithm is as follows:

\begin{split}
\text{Step 1: } & \text{Draw } (\sigma^2)^{(g+1)} \text{ from } q(\sigma^2) \sim \mathcal{IG}\\
\text{Step 2: } & \text{Accept } (\sigma^2)^{(g+1)} \text{ with probability } \alpha \left( (\sigma^2)^{(g+1)}, (\sigma^2)^{(g)} \right)
\end{split}

where

\begin{equation}
\alpha \left( (\sigma)^{(g)}, (\sigma)^{(g+1)} \right) = \min\left( \frac{p\left( C|(\sigma^2)^{(g+1)},S \right)}{p\left( C|(\sigma^2)^{(g)},S \right)},1 \right)
\end{equation}

There are lots of modified models in finance to better model asset/financial instrument prices. For example, in the my final project for the financial calculus course, I have breifly researched on Merton's jump-diffusion model, where the model includes a jump in stock price due to sudden news. In practice, to apply the MCMC algorithm, it is too complex to tackle and has gone beyond my knowledge. A lot more models has been discussed in MCMC Methods for Continuous-Time Financial Econometrics by Michael Johannes and Nicholas Polson and also Bayesian econometrics in finance by Eric Jacquier and Nicholas Polson.

# Reference

Johannes, M., & Polson, N. (2010). MCMC methods for continuous-time financial econometrics. In Handbook of Financial Econometrics: Applications (pp. 1-72). Elsevier.

Jacquier, E., & Polson, N. (2010). Bayesian econometrics in finance.